# AI演習 第11回
### ディープラーニングによる自然言語処理と時系列データ処理 (5)

[実行環境]
Colaboratoryの実行環境は更新されるので確認しておきます。

Python: 3.10.12<br>
TensorFlow: 2.14.0<br>

ランタイムのタイプは、GPUを指定するように注意してください。

In [ ]:
!python -V

Python 3.10.12


In [ ]:
import tensorflow as tf

print(tf.__version__)

2.14.0


# 時系列データと深層学習
* ある現象の時間的な変化を観測して得られる値を対象として、観測時刻と観測値を対応付けたデータ列を時系列データと呼びます。
* 例えば、天気などの気象データ、株価などの金融データ、人や物体などの動作データなどがあります。
* 時系列データを対象とした深層学習には、RNNやLSTMのような，過去の情報を遡って学習できる深層学習モデルを利用します。


## LSTMによる気象予測
* Kerasの公式サイトで公開されているコードを用いて、LSTMによる天気予測を事項してみます。
* 気象データの学習には、ドイツのマックス・プランク研究所で観測されたデータを用いています。

https://www.bgc-jena.mpg.de/wetter/

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

### データの準備と確認

In [ ]:
from zipfile import ZipFile
import os

uri = "https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip"
zip_path = keras.utils.get_file(origin=uri, fname="jena_climate_2009_2016.csv.zip")
zip_file = ZipFile(zip_path)
zip_file.extractall()
csv_path = "jena_climate_2009_2016.csv"

df = pd.read_csv(csv_path)

8年間にわたって、下記の14種類の観測データを記録しています。

"Pressure",
"Temperature",
"Temperature in Kelvin",
"Temperature (dew point)",
"Relative Humidity",
"Saturation vapor pressure",
"Vapor pressure",
"Vapor pressure deficit",
"Specific humidity",
"Water vapor concentration",
"Airtight",
"Wind speed",
"Maximum wind speed",
"Wind direction in degrees"

In [ ]:
print(df.shape)

In [ ]:
print(df)

In [ ]:
titles = [
    "Pressure",
    "Temperature",
    "Temperature in Kelvin",
    "Temperature (dew point)",
    "Relative Humidity",
    "Saturation vapor pressure",
    "Vapor pressure",
    "Vapor pressure deficit",
    "Specific humidity",
    "Water vapor concentration",
    "Airtight",
    "Wind speed",
    "Maximum wind speed",
    "Wind direction in degrees",
]

feature_keys = [
    "p (mbar)",
    "T (degC)",
    "Tpot (K)",
    "Tdew (degC)",
    "rh (%)",
    "VPmax (mbar)",
    "VPact (mbar)",
    "VPdef (mbar)",
    "sh (g/kg)",
    "H2OC (mmol/mol)",
    "rho (g/m**3)",
    "wv (m/s)",
    "max. wv (m/s)",
    "wd (deg)",
]

colors = [
    "blue",
    "orange",
    "green",
    "red",
    "purple",
    "brown",
    "pink",
    "gray",
    "olive",
    "cyan",
]

date_time_key = "Date Time"

In [ ]:
print(df[feature_keys[0]])

#### 確認例題 5-1

"Temperature"のデータを表示してみましょう。

### グラフの描画

In [ ]:
data = df
time_data = data[date_time_key]
key = feature_keys[0]
c = colors[0]
t_data = data[key]
t_data.index = time_data
t_data.head()
ax = t_data.plot(
    color=c,
    title="{} - {}".format(titles[0], key),
    figsize=(8, 3),
    rot=25,
)
ax.legend([titles[0]])
plt.tight_layout()

#### 確認例題 5-2

"Temperature"のデータをグラフで描画してみましょう。

In [ ]:
def show_raw_visualization(data, i):
    time_data = data[date_time_key]
    key = feature_keys[i]
    c = colors[i % (len(colors))]
    t_data = data[key]
    t_data.index = time_data
    t_data.head()
    ax = t_data.plot(
        color=c,
        title="{} - {}".format(titles[i], key),
        figsize=(8, 3),
        rot=25,
    )
    ax.legend([titles[i]])
    plt.tight_layout()

定義した関数show_raw_visualization()を使い、もう一度"Pressure"のデータのグラフを描画していみます。

In [ ]:
show_raw_visualization(df, 0)

#### 確認例題 5-3

残りの"Temperature in Kelvin",
"Temperature (dew point)",
"Relative Humidity",
"Saturation vapor pressure",
"Vapor pressure",
"Vapor pressure deficit",
"Specific humidity",
"Water vapor concentration",
"Airtight",
"Wind speed",
"Maximum wind speed",
"Wind direction in degrees"
のデータについてグラフを描画してみましょう。

### ヒートマップの描画

各データの相関を調べて、ヒートマップで可視化してみます。

まず、corr()関数を使用して、相関を計算します。

In [ ]:
data.corr()

次にヒートマップで可視化します。

In [ ]:
plt.matshow(data.corr())
plt.xticks(range(data.shape[1]), data.columns, fontsize=14, rotation=90)
plt.gca().xaxis.tick_bottom()
plt.yticks(range(data.shape[1]), data.columns, fontsize=14)

cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title("Feature Correlation Heatmap", fontsize=14)
plt.show()


### LSTMでの学習

In [ ]:
split_fraction = 0.715
train_split = int(split_fraction * int(df.shape[0]))
step = 6

past = 720
future = 72
learning_rate = 0.001
batch_size = 256
epochs = 10

In [ ]:
print(
    "The selected parameters are:",
#    ", ".join([titles[i] for i in [0, 1, 5, 7, 8, 10, 11]]),
    ", ".join([titles[i] for i in [0, 1, 5]]),
)
# selected_features = [feature_keys[i] for i in [0, 1, 5, 7, 8, 10, 11]]
selected_features = [feature_keys[i] for i in [0, 1, 5]]

The selected parameters are: Pressure, Temperature, Saturation vapor pressure


In [ ]:
features = df[selected_features]
features.index = df[date_time_key]
features.head()

In [ ]:
features.shape

データを正規化します。

In [ ]:
def normalize(data, train_split):
    data_mean = data[:train_split].mean(axis=0)
    data_std = data[:train_split].std(axis=0)
    return (data - data_mean) / data_std

In [ ]:
features = normalize(features.values, train_split)
features = pd.DataFrame(features)
features.head()

データを訓練用データと検証用データに分割します。

In [ ]:
train_data = features.loc[0 : train_split - 1]
val_data = features.loc[train_split:]

In [ ]:
print(train_data.shape)
print(val_data.shape)

訓練用データx_trainとy_trainを作成します。

In [ ]:
start = past + future
end = start + train_split

#x_train = train_data[[i for i in range(7)]].values
x_train = train_data[[i for i in range(3)]].values
y_train = features.iloc[start:end][[1]]

sequence_length = int(past / step)

In [ ]:
dataset_train = keras.preprocessing.timeseries_dataset_from_array(
    x_train,
    y_train,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch_size,
)

検証用データx_valとy_valを作成します。

In [ ]:
x_end = len(val_data) - past - future

label_start = train_split + past + future

# x_val = val_data.iloc[:x_end][[i for i in range(7)]].values
x_val = val_data.iloc[:x_end][[i for i in range(3)]].values
y_val = features.iloc[label_start:][[1]]

dataset_val = keras.preprocessing.timeseries_dataset_from_array(
    x_val,
    y_val,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch_size,
)


for batch in dataset_train.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

LSTMモデルを作成します。

In [ ]:
inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
lstm_out = keras.layers.LSTM(32)(inputs)
outputs = keras.layers.Dense(1)(lstm_out)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
model.summary()

In [ ]:
history = model.fit(
    dataset_train,
    epochs=2,
    validation_data=dataset_val,
)

気温を予測してみます。

In [ ]:
sample_num = 5
total_error = 0

for x, y in dataset_val.take(sample_num):
  pred = model.predict(x)[0]
  print(y[0].numpy())
  print(pred)
  error = pred - y[0].numpy()
  print("誤差:", error)

  total_error += error

#### 確認例題 5-4

予測誤差の平均を計算してみましょう。

グラフに描画します。

In [ ]:
def show_plot(plot_data, delta, title):
    labels = ["History", "True Future", "Model Prediction"]
    marker = [".-", "rx", "go"]
    time_steps = list(range(-(plot_data[0].shape[0]), 0))
    if delta:
        future = delta
    else:
        future = 0

    plt.title(title)
    for i, val in enumerate(plot_data):
        if i:
            plt.plot(future, plot_data[i], marker[i], markersize=10, label=labels[i])
        else:
            plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
    plt.legend()
    plt.xlim([time_steps[0], (future + 5) * 2])
    plt.xlabel("Time-Step")
    plt.show()
    return

In [ ]:
for x, y in dataset_val.take(5):
    show_plot(
        [x[0][:, 1].numpy(), y[0].numpy(), model.predict(x)[0]],
        12,
        "Single Step Prediction",
    )

#### 確認例題 5-5
* 上記のコードでは、3つの属性"Pressure", "Temperature", "Saturation vapor pressure"を用いて、予測を行うモデルを構築しています。さらに、"Vapor pressure deficit"や"Specific humidity"などの他の属性を加えて予測してみましょう。
* 同様に、5個ほどデータをサンプリングして、予測誤差の平均がどのように変化するかを確認してみましょう。
* 属性のリスト:
 "Pressure",
    "Temperature",
    "Temperature in Kelvin",
    "Temperature (dew point)",
    "Relative Humidity",
    "Saturation vapor pressure",
    "Vapor pressure",
    "Vapor pressure deficit",
    "Specific humidity",
    "Water vapor concentration",
    "Airtight",
    "Wind speed",
    "Maximum wind speed",
    "Wind direction in degrees",

* なお、epoch数は10程度にしてみましょう。

## LSTMによる人間活動の判定モデル
* 加速度センサーやジャイロセンサーから得られる人間行動データ(時系列)を学習し、活動の種類を判定するLSTMモデルを構築します。
* 学習データには、カルフォルニア大学アーバイン校で公開されているUCI HARデータセットを用います。
https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones
* 6種類の行動に分類しています: WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING
* 実験の説明ビデオ https://www.youtube.com/watch?v=XOEN9W05_4A

In [1]:
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot

### データの準備

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip
!unzip "UCI HAR Dataset.zip"

In [36]:
# load a list of files and return as a 3d numpy array
def load_files(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = read_csv(prefix + name, header=None, delim_whitespace=True)
		loaded.append(data.values)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

# load a dataset group, such as train or test
def load_dataset(group, prefix=''):
	filepath = prefix + group + '/Inertial Signals/'
	# load all 9 files as a single array
	filenames = list()
	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_files(filenames, filepath)
	# load class output
	y = read_csv(prefix + group + '/y_'+group+'.txt', header=None, delim_whitespace=True)
	return X, y

In [37]:
	trainX, trainy = load_dataset('train', 'UCI HAR Dataset/')
	print(trainX.shape, trainy.shape)
	# load all test
	testX, testy = load_dataset('test', 'UCI HAR Dataset/')
	print(testX.shape, testy.shape)

	trainy = trainy - 1
	testy = testy - 1
	# one-hotベクトル
	trainy = to_categorical(trainy)
	testy = to_categorical(testy)
	print(trainX.shape, trainy.shape, testX.shape, testy.shape)

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


行動時系列データとして3軸(x, y, z)の加速度だけを使って、行動を予測してみます。

In [38]:
trainX_3 = trainX[:,:,0:3]
n_features = trainX_3.shape[2]

In [39]:
print(trainX_3.shape)
print(n_features)

(7352, 128, 3)
3


In [40]:
testX_3 = testX[:,:,0:3]

### LSTMモデルの構築

In [41]:
n_timesteps, n_features, n_outputs = trainX_3.shape[1], trainX_3.shape[2], trainy.shape[1]
model = Sequential()
model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### 学習と評価

In [42]:
hitsotry = model.fit(trainX_3, trainy, epochs=15, batch_size=64, verbose=0)

In [ ]:
_, accuracy = model.evaluate(testX_3, testy, batch_size=64, verbose=0)

print(accuracy)

#### 確認例題 5-6
エポック数を5, 10, 15, 20, 25, 30のように変化させ、予測精度がどのように変化するかを確認してみましょう。

#### 確認例題 5-7

* 9つ全ての行動時系列データを使って、予測精度がどのように変化するかを確認してみましょう。
* エポック数は同様に5, 10, 15, 20, 25, 30のように変化させてください。

# レポート課題

1. **課題提出用にcolaboratory上で実行するためのページを作成**し、下記の確認例題について実行結果を掲載してください。(**説明文や図など、指定した課題に関係ない内容は削除**してください。)

 * 確認例題 1-10 (week7)
 * 確認例題 4-7 (week10) ※3-7となっていますが誤植です。
 * 確認例題 5-5 (week11)

2. 講義の感想も、同じページ上に記述して下さい。

3. 提出方法と期限
 * ColaboratoryのページのURLを記載して、manabaより提出してください。
 * 提出期限: 12/15(金)

# 参考文献
* Keras公式サイト, https://keras.io/examples/